In [1]:
import transformers
from script.tokenizer import KmerTokenizer

import evaluate
import pandas as pd
from tqdm import tqdm

import argparse
import os
import csv
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import json
import random
import re

import pandas as pd
import re
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, PreTrainedTokenizerFast, LineByLineTextDataset, BertConfig, AutoTokenizer, pipeline
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch

from sklearn.preprocessing import LabelEncoder
import numpy as np
from datasets import Dataset
import os

# data

In [3]:
train_df = pd.read_csv('Data/TeleoSplit/train.csv')
val_df = pd.read_csv('Data/TeleoSplit/val.csv')

In [4]:
train_df

,sequence,phylum,class,order,family,genus,species
0,CCCCAAACAATAAACACACGAAACTAACTAAAATGCTTCGAACCGT...,Chordata,Actinopteri,Scombriformes,Gempylidae,Promethichthys,Promethichthysprometheus
1,CCCCAAGTTCAATATATCCTTCTAACTAAGAAGTTAGCCGAACAAA...,Chordata,Actinopteri,Salmoniformes,Salmonidae,Oncorhynchus,Oncorhynchusmasou
2,CCCCGAACTTAACCCACGAACCTTACCTAAACTGTTTTACATGAAA...,Chordata,Actinopteri,Holocentriformes,Holocentridae,Neoniphon,Neoniphonargenteus
3,CCCCTGTTAAACAGCAACCAATGTAAATAACACAAAAGCACCAACG...,Chordata,Actinopteri,Cypriniformes,Gastromyzontidae,Yaoshania,Yaoshaniapachychilus
4,CCCCAAGCTTCCGGCCCTAATTAATTAAAACCCTACAACTGCAAAG...,Chordata,Actinopteri,Istiophoriformes,Istiophoridae,Istiophorus,Istiophorusplatypterus
...,...,...,...,...,...,...,...
15693,CCCCTGTCAAAATGCAATAAAGATACTTAACACCAAAGCACTGACA...,Chordata,Actinopteri,Cypriniformes,Leuciscidae,Gila,Gilaelegans
15694,CATCAAGTCATAACTTTACACAGTTAACAGACAGATGGGAAAAGTC...,Chordata,Actinopteri,Cypriniformes,Danionidae,Microdevario,Microdevarionanus
15695,CCCCTTATACCTTAAAATTTTAACAAAATTACTAATACCTTAACCT...,Chordata,Actinopteri,Siluriformes,Pimelodidae,Pseudoplatystoma,Pseudoplatystomacorruscans
15696,CCTCAAAAATATTCACCCCTTCTATAAACATACTTCTCCAATAAGA...,Chordata,Chondrichthyes,Carcharhiniformes,Carcharhinidae,Glyphis,Glyphisgarricki


In [5]:
def no_double(df):
    poisson_noDoublon=pd.DataFrame()

    for seq in tqdm(df['sequence'].unique()):

        sub=df[df['sequence']==seq]



        poisson_noDoublon=pd.concat([poisson_noDoublon,sub.sample(1)])
    return poisson_noDoublon

In [6]:
train_df=no_double(train_df)

100%|██████████| 6303/6303 [00:04<00:00, 1287.47it/s]


In [7]:
val_df=no_double(val_df)

100%|██████████| 563/563 [00:00<00:00, 4038.76it/s]


In [8]:
dict_ambigus={'Y':['C','T'],
               'W':['A','T'],
               'M':['A','C'],
               'K':['T','G'],
               'R':['A','G'],
               'S':['C','G'],

               'B':['C','T','G'],
               'V':['A','C','G'],
               'H':['A','C','T'],
               'D':['A','G','T'],
               
               'X':['A','C','G','T'],
               'N':['A','C','G','T'],
               ' ':['']
               }
def clean_nucleotide(df):
    c=0
    df_copy=df.copy()
    sequence=np.array(df['sequence'].values)
    #print(len(sequence))
    for anbigous_nuc in dict_ambigus.keys():
        #print(anbigous_nuc)
        for i, seq in enumerate(sequence) :
            #print("okok")

        
            #print(lambda m:random.choice(dict_ambigus[anbigous_nuc]))
            c+=len(re.findall(seq,anbigous_nuc))
            sequence[i]=re.sub(anbigous_nuc, lambda m:random.choice(dict_ambigus[anbigous_nuc]) , seq)
    df_copy['sequence']=sequence
    print(c)
    return df_copy

In [9]:
train_df=clean_nucleotide(train_df)

0


In [10]:
val_df=clean_nucleotide(val_df)

0


In [11]:
RATIO_UNBALANCED = 0.2

MIN_LEN = 0.9

FREQ_CROP = 0.90

LOW_NUC = 1/40

HIGHT_NUC = 1/20

In [12]:
dict_mutation={'C':['T','G','A'],'T':['C','G','A'],'G':['T','C','A'],'A':['T','G','C']}

def chunk(seq,crop=True,crop_freq=1-FREQ_CROP,min_lenght=MIN_LEN):
    
    if random.random()>=crop_freq:

        sub_lenght=random.randint(int(len(seq)*MIN_LEN),len(seq))

        start_index=random.randint(0,len(seq)-sub_lenght)

        sub_seq=seq[start_index:start_index+sub_lenght]

        return sub_seq
    
    else : return seq

    
def mutationv0(seq,mutation_ratio_min=LOW_NUC,mutation_ration_max=HIGHT_NUC):
    
    lenght=len(seq)
    
    mutation_ration=random.uniform(mutation_ratio_min,mutation_ration_max)

    index_sub=range(lenght)

    index_mutation=random.sample(index_sub,int(mutation_ration*lenght))

    for ind in index_mutation:

        new_nuc=random.choice(dict_mutation[seq[ind]])

        seq=seq[:ind]+new_nuc+seq[ind+1:]

    return seq

In [13]:
def augment_data(df,save_file= 'augment.csv',func_mutation=mutationv0, func_chunk=chunk ,sample_par_class=None, colunm_aug='family'):

    df_manage=df.copy()

    df_manage['synthetic']=[0]*df.shape[0]

    df_manage.to_csv(save_file,index=False)
    
    with open(save_file, mode='a') as csv_file:

        writer = csv.DictWriter(csv_file, fieldnames=df_manage.columns,lineterminator='\n')

        count=df_manage[colunm_aug].value_counts().to_frame()

        count_dict=count.to_dict()

        if sample_par_class ==None:
            max_sample=count.reset_index()['count'][0]
            sample_par_class=int(max_sample*RATIO_UNBALANCED)

        sub_df_need_augment=df_manage[df_manage[colunm_aug].map(df_manage[colunm_aug].value_counts()) < sample_par_class]
        label_need_augment=sub_df_need_augment[colunm_aug].unique()
        #print(sub_df_need_augment)
        #print(label_need_augment)
        for lab in tqdm(label_need_augment):
            

            sub_df=sub_df_need_augment[sub_df_need_augment[colunm_aug]==lab]
            
            augment_number=sample_par_class-count_dict["count"][lab]
            #print(sub_df)
            #print(augment_number)
            for i in range(augment_number):


                row=sub_df.sample(1)
                #print(row.shape)

                for index, r in row.iterrows():

                    

                    
                    #print(sequence)
                    r['sequence'] = func_mutation(func_chunk(r['sequence']))
                    #print(label,sequence)
                    
                    r['synthetic']=1
                        
                    
                    writer.writerow(r.to_dict())

        csv_file.close() 

    return pd.read_csv(save_file)

In [14]:
train_df=augment_data(train_df)

100%|██████████| 366/366 [00:03<00:00, 96.96it/s] 


In [15]:
val_df=augment_data(val_df)

100%|██████████| 207/207 [00:00<00:00, 1200.16it/s]


# model


In [16]:
tokeN=KmerTokenizer(4, trust_remote_code=True)

In [18]:
train_encodings = tokeN(train_df['sequence'].tolist(), truncation=True, max_length=512)
val_encodings = tokeN(val_df['sequence'].tolist(), truncation=True, max_length=512)

In [19]:
# Initialize the label encoder
label_encoder = LabelEncoder()

# Fit the label encoder on the combined data
label_encoder.fit(pd.concat([train_df['family'], val_df['family']]))

# Encode the labels
train_labels = label_encoder.transform(train_df['family'])
val_labels = label_encoder.transform(val_df['family'])

In [20]:
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels
})

val_dataset = Dataset.from_dict({
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask'],
    'labels': val_labels
})

In [21]:
# Create dictionnaries for the class labels
id2label = {i: label for i, label in enumerate(label_encoder.classes_)}
label2id = {label: i for i, label in enumerate(label_encoder.classes_)}

In [22]:
data_collator = DataCollatorWithPadding(tokenizer=tokeN)

In [23]:
config = BertConfig.from_pretrained("zhihan1996/DNABERT-2-117M", 
                                    num_labels=len(id2label), 
                                    max_position_embeddings=514,
                                    id2label=id2label,
                                    label2id=label2id
)

model = AutoModelForSequenceClassification.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True, ignore_mismatched_sizes=True, config=config)

# Verif that the model is correctly configured
model.id2label = id2label
model.label2id = label2id

C:\Users\Auguste\.cache\huggingface\modules\transformers_modules\zhihan1996\DNABERT-2-117M\d064dece8a8b41d9fb8729fbe3435278786931f1\bert_layers.py:126: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at zhihan1996/DNABERT-2-117M and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
from sklearn.metrics import accuracy_score
from transformers import IntervalStrategy


training_args = TrainingArguments(
    output_dir="./test2",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    #auto_find_batch_size=True,
    num_train_epochs=20,
    logging_steps=75,
    eval_on_start=True,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    gradient_accumulation_steps=4,
    eval_accumulation_steps=1,
    logging_first_step=True
)

# class CustomTrainer(Trainer):
#     def _evaluate(self, trial=None, ignore_keys_for_eval=None):
#         # Implement your evaluation logic here
#         # For example, calculate accuracy on a validation set
#         metrics = {"accuracy": 0.9}  # Placeholder for actual evaluation logic
#         return metrics

# Il y a avait un bug avec le compute metrics donc enlevé pour le moment

#def compute_metrics(eval_pred):
 #   predictions, labels = eval_pred
  #  predictions = np.argmax(predictions, axis=1)
   # return accuracy.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokeN,
    data_collator=data_collator,
    #compute_metrics=compute_metrics,
)

In [25]:
# Update the trainer arguments to use your datasets
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # Use F1 score to select the best model
    greater_is_better=True,      # Higher F1 score is better
    push_to_hub=False,
)

# Load F1 metric
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred.predictions, eval_pred.label_ids

    # Add debugging information to inspect shapes
    print(f"Predictions type: {type(predictions)}")
    if isinstance(predictions, tuple):
        predictions = predictions[0]
    print(f"Predictions shape after handling tuple: {predictions.shape}")
    print(f"Labels shape: {labels.shape}")

    # Ensure predictions are 2D
    if predictions.ndim == 2:
        predictions = np.argmax(predictions, axis=1)
    elif predictions.ndim > 2:
        raise ValueError(f"Unexpected predictions shape: {predictions.shape}")

    return f1_metric.compute(predictions=predictions, references=labels, average='macro')

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokeN,
    data_collator=data_collator,
    compute_metrics=compute_metrics,  # Include the compute_metrics function
    #class_weights=class_weights_tensor.to("cuda")  # Uncomment and define this line if you're using class weights
)

c:\Users\Auguste\anaconda3\envs\torch\Lib\site-packages\transformers\training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [26]:
trainer.train()

  0%|          | 0/35940 [00:00<?, ?it/s]

{'loss': 5.9074, 'grad_norm': 2.4107718467712402, 'learning_rate': 9.860879243183083e-06, 'epoch': 0.21}
{'loss': 5.7811, 'grad_norm': 3.810699939727783, 'learning_rate': 9.721758486366167e-06, 'epoch': 0.42}
{'loss': 5.6186, 'grad_norm': 3.016672372817993, 'learning_rate': 9.582637729549249e-06, 'epoch': 0.63}
{'loss': 5.4458, 'grad_norm': 3.546008348464966, 'learning_rate': 9.443516972732332e-06, 'epoch': 0.83}


  0%|          | 0/104 [00:00<?, ?it/s]

Predictions type: <class 'tuple'>
Predictions shape after handling tuple: (1664, 378)
Labels shape: (1664,)
{'eval_loss': 5.292776107788086, 'eval_f1': 0.12000339558283843, 'eval_runtime': 10.321, 'eval_samples_per_second': 161.225, 'eval_steps_per_second': 10.077, 'epoch': 1.0}


TypeError: KmerTokenizer.save_vocabulary() got an unexpected keyword argument 'filename_prefix'